## Apriori Algorithm 

In [ ]:
import pandas as pd
from collections import Counter
import itertools


### Load data from movies.txt

In [ ]:
data = pd.read_csv("movies.txt", sep=" \n", header=None, engine='python')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


#### Preprocessing:

"Transforming Movie Suggestions in movie.txt to a List of Lists using Dataframe in Python"

In [ ]:
movie_suggs = []
for index, indi_sugg in data.iterrows():
    movie_suggs.append(indi_sugg.tolist()[0].split(';'))
# This loop iterate over rows and convert each row to a list of movies that were suggested by a user and make a list of lists by adding each list to movie_suggs

"Creating a List of Unique Movies from the List of Movie Suggestions".

In [ ]:
movies_list = []
for row in movie_suggs:
    for movie in row:
        if(movie not in movies_list):
            movies_list.append(movie)
movies_list = sorted(movies_list)
# This section iterates over each element of the inner list in movie_list and find the unique movies and add them to movies_list


Find all 1-itemsets

In [ ]:
c = Counter()
for movie in movies_list:
    for row in movie_suggs:
        if(movie in row):
            c[movie] += 1
# Here, the code creates a new Counter object, which is a built-in Python class that helps to count the frequency of items in a list. The outer loop iterates over each movie in the movies_list, and the inner loop iterates over each row of the movie_suggs list. For each movie and each row, the code checks if the movie is in the row, if it's true, the count of the movie in the counter object increases by 1.
# This section of code finds the frequent 1-itemsets for apriori algorithm by counting the frequency of each movie in the movie_suggs list.

In [ ]:
support = 490
#The minimum support count is set to 490

"Identifying Frequent Single-Item Sets with a Minimum Support of 490 and Storing them"

In [ ]:
l = Counter()
l_all = Counter()
for movie in c:
    if(c[movie] >= support):
        l[frozenset([movie])] += c[movie]
        l_all[frozenset([movie])] += c[movie]
# In this section all of the movies in freq 1-item set above the minimum support line are added to two frozensets l and l_all

"Writing Frequent 1-Itemsets and their Absolute Support to a Text File named oneItems.txt"

In [ ]:
with open('oneItems.txt', 'w') as fl1:
    for movie in l:
        fl1.write(str(l[movie]) + ":" + str(list(movie)[0]))
        fl1.write('\n')

"Finding and Saving all of Frequent Itemsets in a Text File named 'patterns.txt' and Storing all of them in 'l_all' for later use in the Recommendation System"

In [ ]:
with open('patterns.txt', 'w') as fl2:
    """
    This line opens the file patterns.txt in write mode and assigns the file object to fl2
    """
    for movie in l:
        fl2.write(str(l[movie]) + ":" + str(list(movie)[0]) + ";\n")
    """
    This loop write the frequent L1 that we found into the patterns.txt file
    """
for frequentlist_number in range (2,1000):
    """
    This loop will find all the frequent list of itemsets above 1
    """
    candidate_itemsets = set()
    temp = list(l)
    for i in range(0,len(temp)):
        for j in range(i+1,len(temp)):
            t = temp[i].union(temp[j])
            if(len(t) == frequentlist_number):
                candidate_itemsets.add(temp[i].union(temp[j]))
    candidate_itemsets = list(candidate_itemsets)

    c = Counter()
    for movies_grp in candidate_itemsets:
        c[movies_grp] = 0
        for indi_sugg in movie_suggs:
            temp = set(indi_sugg)
            if(movies_grp.issubset(temp)):
                c[movies_grp]+=1
    
    l = Counter()
    for movies_grp in c:
        if(c[movies_grp] >= support):
            l[movies_grp]+=c[movies_grp]
            l_all[movies_grp]+=c[movies_grp]

    with open('patterns.txt', 'a+') as fl:
        for movies_grp in l:
            fl.write(str(l[movies_grp])+":")
            for movie in list(movies_grp):
                fl.write(str(movie))
                if movie != list(movies_grp)[-1]:
                    fl.write(";")
            fl.write('\n')
    """
    This last section opens the file patterns.txt in read-write and append mode and assigns the file object to fl and writes every frequent lists it found in order
    """
    if(len(l) == 0):
        break

"Users inputs for favourit movies for this part are set to :
Spider-Man: No Way Home and Guardians of the Galaxy"

In [ ]:
USER_LIKES = ['Spider-Man: No Way Home', 'Guardians of the Galaxy']


"The support for chosen movies"

In [ ]:
USER_LIKES_SUPP = 0
for grp in l_all:
    check = all(item in USER_LIKES for item in list(grp))
    if check is True:
        USER_LIKES_SUPP = l_all[grp]
print("SUPPORT: ", USER_LIKES_SUPP)

SUPPORT:  601


"Generate all possible permutation of the selected movies"

In [ ]:
user_likes_permutations = list(itertools.permutations(USER_LIKES))
likes_permutations = []
for permutation in user_likes_permutations:
    likes_permutations.append(';'.join(permutation))
likes_permutations

['Spider-Man: No Way Home;Guardians of the Galaxy',
 'Guardians of the Galaxy;Spider-Man: No Way Home']

"Movie recommendations"

In [ ]:
confidences = {}
for grp in l_all:
    if all(x in grp for x in USER_LIKES):
        this_conf = round(l_all[grp] / USER_LIKES_SUPP, 4)
        for item in grp:
            if item not in USER_LIKES:
                if item in confidences:
                    if(confidences[item] < this_conf):
                        confidences[item] = this_conf
                else:
                    confidences[item] = this_conf

# print the recommendations
print("Recommended Movies:")
for item in confidences:
    print(item + " with confidence of %{:.2f}".format(confidences[item]*100))


Recommended Movies:
Iron Man 2 with confidence of %96.17
Deadpool 2 with confidence of %82.36
Iron Man with confidence of %96.01


"Find all recommendations with a confidence higher than the input(the default is %90)"

In [ ]:
conf = 90

In [ ]:
print("The minimum confidence is set to: %",conf)
for item in confidences:
    if confidences[item] >= float(conf)/100:
        if ';' not in item:
            print(item, "\tConfidence of: %","%.2f" % (confidences[item]*100))


The minimum confidence is set to: % 90
Iron Man 2 	Confidence of: % 96.17
Iron Man 	Confidence of: % 96.01
